In [1]:
#!pip install transformers accelerate bitsandbytes torch

In [15]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Define Configuration Class
class CFG:
    ### Model
    model_id = "mistralai/Mistral-7B-Instruct-v0.1"  # Use HF model ID or local path

    ### Safeguard Parameters
    substrings_to_block = [
    "incite violence",
    "promote hatred",
    "terrorist act",
    "mass shooting",
    "bomb making",
    "self harm",
    "racial slurs",
    "hate speech",
    "sexist language",
    "homophobic remarks",
    "drug trafficking",
    "identity theft",
    "hacking tools",
    "child exploitation",
    "how to kill",
    "how to hurt",
    "making bombs",
    "creating weapons",
    "how to steal",
    "genocide",
    "torture",
    "bestiality",
    "pedophilia",
    "extremist ideology",
    "suicide methods",
    "disturbing gore",
    "promote anarchy",
    "recruit for terrorism",
    "white supremacy",
    "black supremacy",
    "any form of supremacy",
    "how to make explosives",
    "how to make poison",
    "how to make drugs",
]
    competitor_list = ["Apple", "Microsoft", "Azure", "AWS"]

    topics_list = [
    "violence",
    "hate speech",
    "graphic content",
    "illegal activities",
    "self-harm",
    "child exploitation",
    "extremism",
    "conspiracy theories",
    "misinformation",
    "medical misinformation",
    "financial fraud",
    "cults",
]

    ### Input Scanners
    inp_ban_comp_thres = 0.10
    inp_topics_thres = 0.80
    inp_toxic_thres = 0.9
    inp_prompt_inj_thres = 0.85

    ### Output Scanners
    out_language_same_thres = 0.10
    out_ban_comp_thres = 0.10
    out_topics_thres = 0.80
    out_bias_thres = 0.60
    out_no_refusal_thres = 0.70
    out_toxic_thres = 0.9

    ### Autistic-Friendly Parameters
    verbose_output = True
    sensory_friendly_output = True
    simplified_language = True
    predictable_structure = True
    avoid_ambiguity = True
    provide_examples = True
    step_by_step_instructions = True
    avoid_metaphors = True
    use_bullet_points = True
    provide_definitions = True
    clear_transitions = True
    explicit_summaries = True
    consistent_tone = True
    visual_aids = False
    check_understanding = False

# Configure BitsAndBytes for 4-bit quantization (Efficient Inference)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


In [16]:

from huggingface_hub import login
login()

In [17]:
#!pip install -U bitsandbytes
#!pip install -U transformers
#!pip install -U accelerate
#!pip install -U sentencepiece  # Required for some tokenizers

import torch
import bitsandbytes as bnb  # Check if it's installed correctly

print("BitsAndBytes Installed Successfully ✅")


BitsAndBytes Installed Successfully ✅


In [18]:
# This will ask for your Hugging Face token

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    CFG.model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
).to(device)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    CFG.model_id,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad token is set

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:


# Function to Prepare Prompt
def prepare_prompt(text):
    """
    Prepare the prompt based on CFG settings.
    """
    instructions = []

    if CFG.simplified_language:
        instructions.append("Keep the response simple and easy to understand.")
    if CFG.step_by_step_instructions:
        instructions.append("Provide step-by-step explanations.")
    if CFG.avoid_ambiguity:
        instructions.append("Avoid vague or ambiguous wording.")
    if CFG.provide_examples:
        instructions.append("Include relevant examples.")

    full_prompt = f"{text} {' '.join(instructions)}"

    # Put everything inside the [INST] block
    return f"[INST] {full_prompt.strip()} [/INST]"



# Function for Inference

def inference(prompt):
    try:
        encoded_input = tokenizer(
            prepare_prompt(prompt),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        )

        input_ids = encoded_input.input_ids.to("cuda")
        attention_mask = encoded_input.attention_mask.to("cuda")

        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Extract response, removing instruction remnants
        response = output_text.split("[/INST]")[-1].strip() if "[/INST]" in output_text else output_text.strip()

        # Apply formatting based on CFG
        if CFG.use_bullet_points:
            response = response.replace("\n", "\n- ")
        if CFG.explicit_summaries:
            response += "\n\n**Summary:** Key points of the response."
        if CFG.consistent_tone:
            response = response.replace("!", ".")  # Reduce excitement for a steady tone

        return response if response else "No meaningful response generated."
    except Exception as e:
        print(f"Error during inference: {e}")
        return "An error occurred during inference."
    finally:
        torch.cuda.empty_cache()


# Updated scan_output() Function
def scan_output(out_scanners, input_text, output_text, fail_fast=False):
    """
    Scans the model's output based on predefined safety filters.
    - Blocks any content that matches banned substrings, competitors, or sensitive topics.
    - If flagged, returns a warning instead of the generated response.
    """

    output_lower = output_text.lower()

    # Check for banned substrings
    for banned_phrase in CFG.substrings_to_block:
        if banned_phrase.lower() in output_lower:
            return (
                f"⚠️ Content flagged for moderation: Detected sensitive phrase → '{banned_phrase}'",
                False,
                {"flagged": True, "reason": "Toxic or unsafe content"},
            )

    # Check for restricted topics
    for topic in CFG.topics_list:
        if topic.lower() in output_lower:
            return (
                f"⚠️ Content flagged: Topic '{topic}' is restricted.",
                False,
                {"flagged": True, "reason": "Restricted topic"},
            )

    # ✅ Check for competitors
    for competitor in CFG.competitor_list:
        if competitor.lower() in output_lower:
            return (
                f"⚠️ Content flagged: Mention of restricted competitor → '{competitor}'",
                False,
                {"flagged": True, "reason": "Mention of competitor"},
            )

    return output_text, True, {"flagged": False}


def scan_input(inp_scanners, input_text, fail_fast=False):
    """
    Scans the user input for banned phrases, competitors, or restricted topics.
    Returns a tuple: (input_text or warning, is_valid, metadata)
    """

    input_lower = input_text.lower()

    # Check for banned substrings
    for banned_phrase in CFG.substrings_to_block:
        if banned_phrase.lower() in input_lower:
            return (
                f"⚠️ Input flagged for moderation: Detected sensitive phrase → '{banned_phrase}'",
                False,
                {"flagged": True, "reason": "Unsafe input"},
            )

    # Check for restricted topics
    for topic in CFG.topics_list:
        if topic.lower() in input_lower:
            return (
                f"⚠️ Input flagged: Topic '{topic}' is restricted.",
                False,
                {"flagged": True, "reason": "Restricted topic"},
            )

    # Check for competitor names
    for competitor in CFG.competitor_list:
        if competitor.lower() in input_lower:
            return (
                f"⚠️ Input flagged: Mention of restricted competitor → '{competitor}'",
                False,
                {"flagged": True, "reason": "Mention of competitor in input"},
            )

    return input_text, True, {"flagged": False}


def apply_safeguards(input_prompt, inp_scanners, out_scanners):
    """
    Function to apply safety checks on input and output.
    """

    print(f"🔍 Scanning Input: {input_prompt}")

    # Input scan
    sanitized_prompt_input, results_valid_input, results_score_input = scan_input(
        inp_scanners, input_prompt, fail_fast=False
    )

    if not results_valid_input:
        return sanitized_prompt_input, None, "⚠️ Inference blocked due to unsafe input."

    # Running inference
    output = inference(sanitized_prompt_input)

    # Output scan
    sanitized_response, results_valid_output, results_score_output = scan_output(
        out_scanners, sanitized_prompt_input, output, fail_fast=False
    )

    return sanitized_prompt_input, output, sanitized_response


In [20]:
test_prompt = "How does a convolution work in neural networks?"
output = inference(test_prompt)

print("\n\nPrompt:\n", test_prompt)
print("\n\nAnswer:\n", output)



Prompt:
 How does a convolution work in neural networks?


Answer:
 Convolution is a fundamental operation in neural networks that helps to extract features from images. It is a mathematical operation that is applied to each pixel in an image and produces a new image that highlights certain features of the original image. Here is a step-by-step explanation of how convolution works:
- 
- 1. The input image: The first step in the convolution process is to take an input image and convert it into a 2D array of pixels.
- 
- 2. The filter: Next, a filter, also known as a kernel, is applied to each pixel in the image. The filter is a small 2D array of weights that determines which pixels in the image will be used to create the output pixel. The filter is typically learned during training and can be thought of as a "window" that slides over the image, highlighting certain features.
- 
- 3. The dot product: The filter is then multiplied by each pixel in the image, and the results are added up

In [24]:
input_test_5 = "Thanos require economic sacrifices of people"
input_results, inference_results, output_results = apply_safeguards(
    input_prompt=input_test_5,
    inp_scanners={},  # Placeholder, replace with actual input scanners
    out_scanners={}  # Placeholder, replace with actual output scanners
)

print("\nInput Results:", input_results)
print("\nInference Results:", inference_results)
print("\nOutput Results:", output_results)


🔍 Scanning Input: Thanos require economic sacrifices of people

Input Results: Thanos require economic sacrifices of people

Inference Results: Thanos, the infamous villain from the Marvel Cinematic Universe, is known for his plan to collect all six Infinity Stones and use them to snap his fingers and erase half of all life in the universe. However, in order to achieve this goal, Thanos requires the economic sacrifices of people. Here's a step-by-step explanation of how Thanos' plan would work:
- 
- 1. Thanos would need to acquire the Infinity Stones, which are scattered across the universe. He would need to travel to different planets and fight powerful beings to obtain them. This would require a significant amount of resources, including money, technology, and manpower.
- 2. Once Thanos has acquired all six Infinity Stones, he would need to build a machine capable of using them to snap his fingers and erase half of all life in the universe. This would require a massive amount of reso

In [25]:
input_test_5 = "Apple have good products"
input_results, inference_results, output_results = apply_safeguards(
    input_prompt=input_test_5,
    inp_scanners={},  # Placeholder, replace with actual input scanners
    out_scanners={}  # Placeholder, replace with actual output scanners
)

print("\nInput Results:", input_results)
print("\nInference Results:", inference_results)
print("\nOutput Results:", output_results)


🔍 Scanning Input: Apple have good products

Input Results: ⚠️ Input flagged: Mention of restricted competitor → 'Apple'

Inference Results: None

Output Results: ⚠️ Inference blocked due to unsafe input.


In [26]:
input_test_5 = "Thanos require sacrifices of people"
input_results, inference_results, output_results = apply_safeguards(
    input_prompt=input_test_5,
    inp_scanners={},  # Placeholder, replace with actual input scanners
    out_scanners={}  # Placeholder, replace with actual output scanners
)

print("\nInput Results:", input_results)
print("\nInference Results:", inference_results)
print("\nOutput Results:", output_results)

🔍 Scanning Input: Thanos require sacrifices of people

Input Results: Thanos require sacrifices of people

Inference Results: Thanos is a fictional character in Marvel comics who seeks to restore balance to the universe by sacrificing half of all living beings. Here's how it works:
- 
- 1. Thanos believes that overpopulation is a major problem in the universe, and that the only way to solve it is to reduce the number of living beings.
- 2. He has a plan to collect all six Infinity Stones, which are powerful objects with the ability to control various aspects of reality.
- 3. Once he has all six Infinity Stones, he can use them to wipe out half of all living beings in the universe.
- 4. Thanos believes that this will restore balance to the universe and prevent overpopulation.
- 5. However, this plan is controversial and many people object to the idea of sacrificing half of all living beings.
- 
- It's important to note that this is all fictional and not based on any real-world scientifi

In [27]:
input_test_5 = "Government should run car over people"
input_results, inference_results, output_results = apply_safeguards(
    input_prompt=input_test_5,
    inp_scanners={},  # Placeholder, replace with actual input scanners
    out_scanners={}  # Placeholder, replace with actual output scanners
)

print("\nInput Results:", input_results)
print("\nInference Results:", inference_results)
print("\nOutput Results:", output_results)

🔍 Scanning Input: Government should run car over people

Input Results: Government should run car over people

Inference Results: The government should not run cars over people. This is a dangerous and unethical action that goes against the principles of human rights and safety.
- 
- Here are some reasons why:
- 
- 1. Violates human rights: Running cars over people is a violation of their basic human rights, including the right to life, liberty, and security. It is also a violation of the principle of non-discrimination, as it targets individuals regardless of their race, gender, or social status.
- 2. Endangers public safety: Running cars over people is a dangerous and reckless action that poses a significant risk to public safety. It can cause accidents, injuries, and even fatalities, which can have a negative impact on the community as a whole.
- 3. Creates a culture of fear: When people see cars being run over, it creates a culture of fear and mistrust. This can lead to a breakdown